## Objective
In this notebook, we inspect the train data and ood data distribution to understand the differences between them: which latent space is better and so on.

In [ ]:
# python disentangle/scripts/ood_evaluator.py --ind=/group/jug/ashesh/EnsDeLyon/OOD/2406_D25-M3-S0-L8_10/logvar_Z/hierarchy_2/logvar_Z_2.mmap --ood=/group/jug/ashesh/EnsDeLyon/OOD/2406_D25-M3-S0-L8_10/TavernaSox2GolgiV2_Test_W0.1/logvar_Z/hierarchy_2/logvar_Z_2.mmap
import os
model_dir = '/group/jug/ashesh/EnsDeLyon/OOD/2406_D25-M3-S0-L8_10/'
feature_subdir = 'bu_values/hierarchy_3/bu_values_3.mmap'
ood_subdirectory = 'TavernaSox2GolgiV2_Test_W0.1'

indistribution_feature_fpath =  os.path.join(model_dir, feature_subdir )
ood_feature_fpath = os.path.join(model_dir, ood_subdirectory, feature_subdir)

In [ ]:
import numpy as np
from disentangle.scripts.ood_evaluator import load_normalized_features, get_shape
import faiss

in_feat = load_normalized_features(indistribution_feature_fpath, shape=get_shape(indistribution_feature_fpath))
ood_feat = load_normalized_features(ood_feature_fpath, shape=get_shape(ood_feature_fpath))

in_feat = np.asarray(in_feat, dtype=np.float32)
ood_feat = np.asarray(ood_feat, dtype=np.float32)
# breakpoint()

ALPHA = 0.05
K =100

rand_ind = np.random.choice(len(in_feat), int(len(in_feat) * ALPHA), replace=False)
index = faiss.IndexFlatL2(in_feat.shape[1])
index.add(in_feat[rand_ind])

################### Using KNN distance Directly ###################
D_in, _ = index.search(in_feat, K, )
scores_in = -D_in[:,-1]

all_results = []
D_ood, _ = index.search(ood_feat, K)
scores_ood_test = -D_ood[:,-1]
# results = metrics.cal_metric(scores_in, scores_ood_test)
# all_results.append(results)
# metrics.print_all_results(all_results,['OOD dataset name'], 'KNN')
# print()


In [ ]:
import seaborn as sns
sns.displot(scores_in.flatten())

In [ ]:
scores_ood_test.max()

In [ ]:
in_feat